<a href="https://colab.research.google.com/github/DolevD42/deep_networks_project/blob/main/LeNet_d1_d2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import matplotlib.pyplot as plt


# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# results needed:

error_rates = []


# define consts:
learning_rate = 0.028
decay = 9.5e-4
batch_size = 100
kernel_size = 5
pool_size = 2

d1_values = [1,2,3,6,12,18]
d2_values = [2,5,8,16,32,48]
momentum_values = [0.85,0.85,0.905,0.910,0.915,0.95]
epochs_values = [240,240,220,280,240,280]

for i in range(len(d1_values)):
    epochs = epochs_values[i]
    momentum = momentum_values[i]
    d1 = d1_values[i]
    d2 = d2_values[i]
    # Define model
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(3,d1,kernel_size)
            self.rel = nn.ReLU()
            self.pool = nn.MaxPool2d(pool_size, pool_size)
            self.conv2 = nn.Conv2d(d1,d2,kernel_size)
            self.flatten = nn.Flatten()
            self.network_steps_once_flattan = nn.Sequential(
                nn.Linear(5*5*d2, 120),
                nn.ReLU(),
                nn.Linear(120, 84),
                nn.ReLU(),
                nn.Linear(84, 10)
            )

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = self.pool(x)
            x = F.relu(self.conv2(x))
            x = self.pool(x)
            x = self.flatten(x)
            x = self.network_steps_once_flattan(x)
            return x

    model = NeuralNetwork().to(device)
    print(model)

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=momentum,weight_decay = decay)


    def train(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()

            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()
            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    def test(dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        return correct

    for t in range(epochs):
        acc_values = []
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        acc_values.append(test(test_dataloader, model, loss_fn))
    plt.plot(acc_values)
    plt.title("d1="+str(d1))
    plt.xlabel("epochs")
    plt.ylabel("Accuracy")
    error_rates.append(1-acc_values[-1])
    print("Done!")
print(error_rates)
plt.plot(d1_values,error_rates)
plt.xlabl("d1")
plt.ylabel("error_rate")

100%|██████████| 170498071/170498071 [00:06<00:00, 27821151.22it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
Using cpu device
NeuralNetwork(
  (conv1): Conv2d(3, 1, kernel_size=(5, 5), stride=(1, 1))
  (rel): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(1, 2, kernel_size=(5, 5), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (network_steps_once_flattan): Sequential(
    (0): Linear(in_features=50, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.312677  [  100/50000]
loss: 2.157707  [10100/50000]
loss: 2.063633  [20100/50000]
loss: 1.880225  [30100/50000]
loss: 1.769162  [40100/50000]
Test Error: 
 Accuracy: 32.6%, Avg loss: 1.862744 

Epoch 2
-------------------------------
loss: 1.887659  [  100/50000]
loss: 1.789782  [10100/50000]
loss: 1.8170